In [1]:
import tensorflow as tf
import numpy as np
from numpy import genfromtxt
from sklearn import preprocessing
raw_data = genfromtxt('1800_recent.csv', delimiter=',', skip_header=1)
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
normalized_x = preprocessing.normalize(raw_data,axis=0)

In [3]:
print(normalized_x[1])
print(raw_data[1])
# print(normalized_x[-1])
# print(raw_data[-1])
# print(normalized_x[-2])
# print(raw_data[-2])

[  8.79268990e-04   8.91253515e-04   8.82577251e-04   8.87550527e-04
   2.30543006e-06   2.92056079e-05]
[  7.68999818   7.63542      7.64         7.68999818  52.62395636
   6.88413339]


In [4]:
# raw_data= raw_data[:,:4]
# normalized_x = raw_data

In [5]:
seq_length = 7
data_dim=6
hidden=10
batch_size=1
output_dim=1
learning_rate=0.01
iterations = 5000

In [6]:
xy = normalized_x
data_x = xy
data_y = xy[:,[3]]

train_size = int(len(data_y)*0.7)
test_size = len(data_y)-train_size

dataX=[]
dataY=[]

for i in range(0,len(data_y)-seq_length):
    _x = data_x[i:i+seq_length]
    _y = data_y[i+seq_length]
#     print(_x, "=> ",_y)
    dataX.append(_x)
    dataY.append(_y)

In [7]:
print(xy[:,3])
print(xy[:,[3]])

[ 0.0008822   0.00088755  0.00088755 ...,  0.04216774  0.04251934
  0.04248985]
[[ 0.0008822 ]
 [ 0.00088755]
 [ 0.00088755]
 ..., 
 [ 0.04216774]
 [ 0.04251934]
 [ 0.04248985]]


In [8]:
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden, state_is_tuple=True, activation=tf.tanh)
cell = tf.contrib.rnn.GRUCell(num_units=hidden, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
# outputs, _states = tf.nn.static_rnn(cell, X, dtype=tf.float32)

print(outputs)

Tensor("rnn/transpose:0", shape=(?, 7, 10), dtype=float32)


In [9]:
print(outputs.shape)
print(outputs)
print(outputs[:,-1,:].shape, outputs[:,-1].shape)

(?, 7, 10)
Tensor("rnn/transpose:0", shape=(?, 7, 10), dtype=float32)
(?, 10) (?, 10)


In [10]:
Y_pred = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim, activation_fn=None)

In [11]:
loss = tf.reduce_sum(tf.square(Y_pred- Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optimizer.minimize(loss)

In [12]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [13]:
for i in range(iterations):
    _,step_loss = sess.run([train,loss], feed_dict={X:trainX, Y:trainY})
    
    if i%100 == 0:
        print(i, "loss : ",step_loss)

0 loss :  611.887
100 loss :  19.9317
200 loss :  19.636
300 loss :  19.6028


KeyboardInterrupt: 

In [ ]:
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets- predictions)))

In [ ]:
test_prediction = sess.run(Y_pred, feed_dict={X:testX})
rmse_val = sess.run(rmse, feed_dict={targets:testY, predictions:test_prediction})

In [ ]:
print("rmse :", rmse_val)

In [ ]:
plt.plot(testY, label='real_data')
plt.plot(test_prediction, label='prediction')
# plt.subplot(x=len(test_prediction), y=test_prediction, label='prediction')
plt.legend()
plt.xlabel("Time period")
plt.ylabel("Stock Price")
plt.show()

In [ ]:
print(testY.shape, test_prediction.shape)